# VPR Pipeline Demo (Colab-ready)
This notebook runs a quick demo of the VPR pipeline.


## Setup Environment
### Install dependencies (run in Colab)
Run the following cell to install required libraries.

In [ ]:
# ✅ Clone & install everything
!git clone https://github.com/SabraHashemi/VPR-Pipeline-Final.git
%cd VPR-Pipeline-Final
!pip install -r requirements.txt
!bash scripts/setup_colab.sh
!bash scripts/download_matcher_weights.sh

### Download Datasets

!bash scripts/download_datasets.sh

###  Install backbones

In [ ]:
! bash scripts/install_backbones.sh


## (Optional) Install heavy matchers and MixVPR
If you plan to use Full mode, install MixVPR and download weights as instructed in the repo.

## Run the light pipeline (fast)

In [ ]:
!python3 src/cli.py pipeline --db example_data/database --query example_data/queries --backbone resnet50 --mode light

In [ ]:
!python3 src/cli.py pipeline \
  --db datasets/GSV-XS/database \
  --query datasets/GSV-XS/queries \
  --backbone netvlad \
  --mode full \
  --k 10 \
  --use_gpu \
  --run_name "GSV-XS_NetVLAD_Full_GPU"

## Visualize Results

### Inspect results

In [ ]:
import json, os
print(open('outputs/pipeline_results.json').read())
print('Matches folder:', os.listdir('matches') if os.path.exists('matches') else 'none')

In [ ]:
import json, matplotlib.pyplot as plt, os
from PIL import Image

# Load pipeline results
with open('outputs/pipeline_results.json','r') as f:
    results = json.load(f)

print("Results Summary:")
print(json.dumps(results, indent=2))

# Plot recall@K comparison
pre = results['recall_pre']
post = results['recall_post']
labels = list(pre.keys())
pre_vals = [pre[k] for k in labels]
post_vals = [post[k] for k in labels]

plt.figure(figsize=(6,4))
plt.plot(labels, pre_vals, marker='o', label='Before re-ranking')
plt.plot(labels, post_vals, marker='s', label='After re-ranking')
plt.title('Recall@K Comparison')
plt.xlabel('Metric')
plt.ylabel('Recall')
plt.legend()
plt.grid(True)
plt.show()

# Show one example of match visualization
example = sorted(os.listdir('matches'))[0] if os.path.exists('matches') and os.listdir('matches') else None
if example:
    display(Image.open(os.path.join('matches', example)))
else:
    print("No match visualizations found.")


### Weights & Biases Logging

In [ ]:

# 1. Login:
#     !wandb login
# 2. Then rerun the pipeline command with logging enabled:
#     !python3 src/cli.py pipeline --db example_data/database --query example_data/queries --mode full --use_gpu --run_name colab_wandb_test
